# Transformer Modeli ile CPU Tahminleri


In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import time
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Adım 1: Kurulum ve Cihaz Belirleme

In [12]:
# Tüm hesaplamaların yapılacağı cihazı belirle (GPU varsa GPU, yoksa CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Hesaplamalar için kullanılacak cihaz: {device}')

Hesaplamalar için kullanılacak cihaz: cuda


## Adım 2: Veri Yükleme ve Ön İşleme

In [13]:
# İşlenmiş veriyi oku
df = pd.read_csv('genel_sunucu_yuku_tum_metrikler.csv', index_col='timestamp')
df.index = pd.to_datetime(df.index, unit='s')

# Tahmin için 3 sütunu da kullan
ts_data = df[['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']].values

# Veriyi 0-1 arasına ölçeklendir
scaler = MinMaxScaler(feature_range=(0, 1))
ts_data_scaled = scaler.fit_transform(ts_data)

# Veriyi eğitim ve test setlerine ayır
train_size = int(len(ts_data_scaled) * 0.8)
train_data, test_data = ts_data_scaled[:train_size], ts_data_scaled[train_size:]

print("Veri yüklendi, ölçeklendi ve bölündü.")

Veri yüklendi, ölçeklendi ve bölündü.


## Adım 3: Zaman Serisi Dizileri Oluşturma

In [14]:
def create_sequences(data, lookback_window):
    """Veriden LSTM'in anlayacağı sıralı diziler oluşturur."""
    X, y = [], []
    for i in range(len(data) - lookback_window):
        feature = data[i:(i + lookback_window)]
        target = data[i + lookback_window]
        X.append(feature)
        y.append(target)
        
    # PyTorch'a vermeden önce listeleri tek bir NumPy dizisine çeviriyoruz.
    return torch.FloatTensor(np.array(X)), torch.FloatTensor(np.array(y))


lookback = 12  # 1 saatlik veri

X_train, y_train = create_sequences(train_data, lookback)
X_test, y_test = create_sequences(test_data, lookback)

print("Eğitim verisi şekli (X, y):", X_train.shape, y_train.shape)
print("Test verisi şekli (X, y):", X_test.shape, y_test.shape)

Eğitim verisi şekli (X, y): torch.Size([6899, 12, 3]) torch.Size([6899, 3])
Test verisi şekli (X, y): torch.Size([1716, 12, 3]) torch.Size([1716, 3])


## Adım 4: Transformer Modelini Tanımlama ve GPU'ya Taşıma

In [15]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

In [16]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim=3, d_model=64, nhead=4, num_encoder_layers=2, dim_feedforward=128, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.decoder = nn.Linear(d_model, input_dim)

    def forward(self, src):
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output[:, -1, :])
        return output

In [17]:
model = TransformerModel().to(device) # Modeli oluşturup doğrudan GPU'ya taşıyoruz

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Transformer Model Mimarisi:")
print(model)

Transformer Model Mimarisi:
TransformerModel(
  (encoder): Linear(in_features=3, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=64, out_features=3, bias=True)
)


## Adım 5: Modeli GPU Üzerinde Eğitme

In [18]:
epochs = 15

print(f"Model {epochs} epoch boyunca '{device}' üzerinde eğitiliyor...")
start_time = time.time()
model.train()

for i in range(epochs):
    for seq, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        
        # Veriyi her adımda GPU'ya gönder
        seq_gpu = seq.unsqueeze(0).to(device)
        labels_gpu = labels.to(device)
        
        y_pred = model(seq_gpu)
        
        single_loss = loss_function(y_pred.squeeze(0), labels_gpu)
        single_loss.backward()
        optimizer.step()


    print(f'Epoch: {i+1:3} loss: {single_loss.item():10.8f}')

end_time = time.time()
print(f"Eğitim tamamlandı. Süre: {end_time - start_time:.2f} saniye")

Model 15 epoch boyunca 'cuda' üzerinde eğitiliyor...
Epoch:   1 loss: 0.00184379
Epoch:   2 loss: 0.00071774
Epoch:   3 loss: 0.00196255
Epoch:   4 loss: 0.00063375
Epoch:   5 loss: 0.00082121
Epoch:   6 loss: 0.00066598
Epoch:   7 loss: 0.00063935
Epoch:   8 loss: 0.00041635
Epoch:   9 loss: 0.00073865
Epoch:  10 loss: 0.00070712
Epoch:  11 loss: 0.00079325
Epoch:  12 loss: 0.00079790
Epoch:  13 loss: 0.00076885
Epoch:  14 loss: 0.00061531
Epoch:  15 loss: 0.00052079
Eğitim tamamlandı. Süre: 248.61 saniye


## Adım 6: GPU ile Tahmin Yapma ve Sonuçları Değerlendirme

In [19]:
model.eval()
print("Test verisi üzerinde tahminler yapılıyor...")

test_predictions = []
with torch.no_grad():
    for seq, labels in zip(X_test, y_test):
        # Tahmin için veriyi GPU'ya gönder
        seq_gpu = seq.unsqueeze(0).to(device)
        y_pred = model(seq_gpu)
        # Sonucu CPU'ya geri alıp numpy'a çeviriyoruz
        test_predictions.append(y_pred.squeeze(0).cpu().numpy())

print("Tahminler tamamlandı.")

# Tahminleri ve gerçek değerleri orijinal ölçeğine geri çevir
predictions_array = np.array(test_predictions)
predictions_original_scale = scaler.inverse_transform(predictions_array)
y_test_original_scale = scaler.inverse_transform(y_test.numpy())

# MAPE için özel fonksiyonumuzu tanımlayalım
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Sıfıra bölme hatasını önlemek için küçük bir epsilon değeri ekleyelim
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

# Tahmin ettiğimiz sütunların isimleri
target_columns = ['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']

print("--- Transformer Model Performans Metrikleri ---")
# Her bir metrik için ayrı ayrı hata hesaplayalım
for i, column in enumerate(target_columns):
    y_true_col = y_test_original_scale[:, i]
    y_pred_col = predictions_original_scale[:, i]
    
    # Tüm metrikleri hesapla
    mae = mean_absolute_error(y_true_col, y_pred_col)
    rmse = np.sqrt(mean_squared_error(y_true_col, y_pred_col))
    r2 = r2_score(y_true_col, y_pred_col)
    mape = mean_absolute_percentage_error(y_true_col, y_pred_col)
    
    print(f"\n--- Metrik: '{column}' ---")
    print(f"  Ortalama Mutlak Hata (MAE): {mae:.2f}")
    print(f"  Kök Ortalama Kare Hata (RMSE): {rmse:.2f}")
    print(f"  R-Kare (R²): {r2:.2f}")
    print(f"  Ortalama Mutlak Yüzde Hata (MAPE): {mape:.2f}%")

Test verisi üzerinde tahminler yapılıyor...
Tahminler tamamlandı.
--- Transformer Model Performans Metrikleri ---

--- Metrik: 'avg_min_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.19
  Kök Ortalama Kare Hata (RMSE): 0.23
  R-Kare (R²): -0.06
  Ortalama Mutlak Yüzde Hata (MAPE): 3.01%

--- Metrik: 'avg_max_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.47
  Kök Ortalama Kare Hata (RMSE): 0.59
  R-Kare (R²): -0.04
  Ortalama Mutlak Yüzde Hata (MAPE): 2.37%

--- Metrik: 'avg_avg_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.30
  Kök Ortalama Kare Hata (RMSE): 0.35
  R-Kare (R²): -0.06
  Ortalama Mutlak Yüzde Hata (MAPE): 3.04%


## Adım 7: Sonuçları Görselleştirme

In [20]:
# Sonuçları görselleştir
fig = go.Figure()

for i, column in enumerate(target_columns):
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=y_test_original_scale[:, i],
        mode='lines',
        name=f'Gerçek {column}'
    ))
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=predictions_original_scale[:, i],
        mode='lines',
        name=f'Tahmin {column}',
        line=dict(dash='dash')
    ))

fig.update_layout(
    title='CPU Metrikleri: Gerçek Değerler vs. Transformer Tahminleri (GPU)',
    xaxis_title='Zaman',
    yaxis_title='Ortalama CPU Yükü (%)'
)

fig.show()